# Initial Read/JSON Conversions

In [1]:
import pandas as pd
from pandas.io.json import loads, json_normalize
    
df = pd.read_csv('./all/train.csv', dtype={'fullVisitorId': str})

* Function to parse json columns

In [2]:
# CLEAN STEP 1
def parse_json_cols(df):
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

    for col in json_cols:
        df_json = json_normalize(df[col].apply(loads), sep='/')
        df_json.columns = [(col + '/' + c) for c in df_json.columns]
        df.drop(columns=col, inplace=True)
        df = pd.concat([df, df_json], axis=1)
    
    return df

In [3]:
df = parse_json_cols(df)

totals_cols = list(filter(lambda c : c.startswith('totals'), df.columns))
geoNetwork_cols = list(filter(lambda c : c.startswith('geoNetwork'), df.columns))
device_cols = list(filter(lambda c : c.startswith('device'), df.columns))
trafficSource_cols = list(filter(lambda c : c.startswith('trafficSource'), df.columns))

# Examine JSON Data

### totals/
* These fields should be treated as numeric values, not objects
* Cast them to fix this issue

In [4]:
# CLEAN STEP 2

def cast_totals(df):
    totals_cols = list(filter(lambda c : c.startswith('totals'), df.columns))
    for i in totals_cols:
        df[i] = df[i].astype(float)
    return df

In [5]:
df = cast_totals(df)

In [6]:
df[totals_cols].describe()

,totals/bounces,totals/hits,totals/newVisits,totals/pageviews,totals/transactionRevenue,totals/visits
count,450630.0,903653.000000,703060.0,903553.000000,1.151500e+04,903653.0
mean,1.0,4.596538,1.0,3.849764,1.337448e+08,1.0
std,0.0,9.641437,0.0,7.025274,4.482852e+08,0.0
min,1.0,1.000000,1.0,1.000000,1.000000e+04,1.0
25%,1.0,1.000000,1.0,1.000000,2.493000e+07,1.0
50%,1.0,2.000000,1.0,1.000000,4.945000e+07,1.0
75%,1.0,4.000000,1.0,4.000000,1.076550e+08,1.0
max,1.0,500.000000,1.0,469.000000,2.312950e+10,1.0


---
* The visits, newVisits, and bounces columns only has 1 unique value

In [7]:
# CLEAN STEP 3
def drop_visits(df):
    df.drop(columns=['totals/visits', 'totals/newVisits', 'totals/bounces'], inplace=True)
    return df

In [8]:
df = drop_visits(df)

totals_cols = list(filter(lambda c : c.startswith('totals'), df.columns))
df[totals_cols].describe()

,totals/hits,totals/pageviews,totals/transactionRevenue
count,903653.000000,903553.000000,1.151500e+04
mean,4.596538,3.849764,1.337448e+08
std,9.641437,7.025274,4.482852e+08
min,1.000000,1.000000,1.000000e+04
25%,1.000000,1.000000,2.493000e+07
50%,2.000000,1.000000,4.945000e+07
75%,4.000000,4.000000,1.076550e+08
max,500.000000,469.000000,2.312950e+10


---
### geoNetwork/

In [9]:
df[geoNetwork_cols].describe()

,geoNetwork/city,geoNetwork/cityId,geoNetwork/continent,geoNetwork/country,geoNetwork/latitude,geoNetwork/longitude,geoNetwork/metro,geoNetwork/networkDomain,geoNetwork/networkLocation,geoNetwork/region,geoNetwork/subContinent
count,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653
unique,649,1,6,222,1,1,94,28064,1,376,23
top,not available in demo dataset,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Northern America
freq,508229,903653,450377,364744,903653,903653,508229,244881,903653,508229,390657


* most columns have the majority (or all of their values) as "not available in demo dataset" or "(not set)"
        * city, cityId, latitude, longitude, metro, networkLocation, region, networkDomain
---
**Dropping listed columns:**

In [10]:
# CLEAN STEP 4
def drop_geo_cols(df):
    df.drop(columns=['geoNetwork/city', 'geoNetwork/cityId', 'geoNetwork/latitude', 'geoNetwork/longitude', 
                 'geoNetwork/metro', 'geoNetwork/networkLocation','geoNetwork/region', 'geoNetwork/networkDomain'], 
                 inplace=True)
    return df

In [11]:
df = drop_geo_cols(df)

geoNetwork_cols = list(filter(lambda c : c.startswith('geoNetwork'), df.columns))
df[geoNetwork_cols].describe()

,geoNetwork/continent,geoNetwork/country,geoNetwork/subContinent
count,903653,903653,903653
unique,6,222,23
top,Americas,United States,Northern America
freq,450377,364744,390657


---
### device/
* Again, mosts columns only have one value: "not available in demo dataset"
        * browserSize, browserVersion, flashVersion, language, mobileDeviceBranding, mobileDeviceInfo, mobileDeviceMarketingName, mobileDeviceModel, mobileInputSelector, operatingSystemVersion, screenColors, screenResolution

In [12]:
df[device_cols].describe()

,device/browser,device/browserSize,device/browserVersion,device/deviceCategory,device/flashVersion,device/isMobile,device/language,device/mobileDeviceBranding,device/mobileDeviceInfo,device/mobileDeviceMarketingName,device/mobileDeviceModel,device/mobileInputSelector,device/operatingSystem,device/operatingSystemVersion,device/screenColors,device/screenResolution
count,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653
unique,54,1,1,3,1,2,1,1,1,1,1,1,20,1,1,1
top,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset
freq,620364,903653,903653,664479,903653,664530,903653,903653,903653,903653,903653,903653,350072,903653,903653,903653


---
**Dropping listed columns:**

In [13]:
# CLEAN STEP 5
def drop_device_cols(df):
    df.drop(columns=['device/browserSize', 'device/browserVersion', 'device/flashVersion', 'device/language',
                 'device/mobileDeviceBranding', 'device/mobileDeviceInfo', 'device/mobileDeviceMarketingName', 'device/mobileDeviceModel',
                 'device/mobileInputSelector', 'device/operatingSystemVersion', 'device/screenColors', 'device/screenResolution'],
                 inplace=True)
    return df

In [14]:
df = drop_device_cols(df)

device_cols = list(filter(lambda c : c.startswith('device'), df.columns))
df[device_cols].describe()

,device/browser,device/deviceCategory,device/isMobile,device/operatingSystem
count,903653,903653,903653,903653
unique,54,3,2,20
top,Chrome,desktop,False,Windows
freq,620364,664479,664530,350072


---
### trafficSource/
* A few columns here have majority "not available in demo dataset", "(not set)", "(not provided)" values:
        * adwordsClickInfo/criteriaParameters, campaign, keyword
* Column "campaignCode" has only one entry. "gclId" has only 70.

In [15]:
df[trafficSource_cols].describe()

,trafficSource/adContent,trafficSource/adwordsClickInfo/adNetworkType,trafficSource/adwordsClickInfo/criteriaParameters,trafficSource/adwordsClickInfo/gclId,trafficSource/adwordsClickInfo/isVideoAd,trafficSource/adwordsClickInfo/page,trafficSource/adwordsClickInfo/slot,trafficSource/campaign,trafficSource/campaignCode,trafficSource/isTrueDirect,trafficSource/keyword,trafficSource/medium,trafficSource/referralPath,trafficSource/source
count,10946,21460,903653,21561,21460,21460,21460,903653,1,274005,400724,903653,330941,903653
unique,44,2,1,17774,1,8,2,10,1,1,3659,7,1475,380
top,Google Merchandise Collection,Google Search,not available in demo dataset,Cj0KEQjwmIrJBRCRmJ_x7KDo-9oBEiQAuUPKMufMpuG3Zd...,False,1,Top,(not set),11251kjhkvahf,True,(not provided),organic,/,google
freq,5122,21453,903653,70,21460,21362,20956,865347,1,274005,366363,381561,75523,400788


---
**Dropping listed columns:**

In [16]:
# CLEAN STEP 6
def drop_trafficSource_cols(df):
    df.drop(columns=['trafficSource/adwordsClickInfo/criteriaParameters', 'trafficSource/campaign', 'trafficSource/keyword',
                     'trafficSource/adwordsClickInfo/gclId'], inplace=True)
    if 'trafficSource/campaignCode' in df.columns:
        df.drop(columns=['trafficSource/campaignCode'], inplace=True)
    return df

In [17]:
df = drop_trafficSource_cols(df)

trafficSource_cols = list(filter(lambda c : c.startswith('trafficSource'), df.columns))
df[trafficSource_cols].describe()

,trafficSource/adContent,trafficSource/adwordsClickInfo/adNetworkType,trafficSource/adwordsClickInfo/isVideoAd,trafficSource/adwordsClickInfo/page,trafficSource/adwordsClickInfo/slot,trafficSource/isTrueDirect,trafficSource/medium,trafficSource/referralPath,trafficSource/source
count,10946,21460,21460,21460,21460,274005,903653,330941,903653
unique,44,2,1,8,2,1,7,1475,380
top,Google Merchandise Collection,Google Search,False,1,Top,True,organic,/,google
freq,5122,21453,21460,21362,20956,274005,381561,75523,400788


# Examining Remaining Columns (Non JSON)
* Dates are not parsed
* visitStartTime is in POSIX time
    * visitStartTime makes Date redundant (more information encoded)
* Looking only at qualitative data shows socialEngagementType only has 1 unique value (therefore contributes no information)

In [18]:
others = [c for c in df.columns if c not in (totals_cols + geoNetwork_cols + device_cols + trafficSource_cols)]
df[others].head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600


In [19]:
df[['channelGrouping', 'socialEngagementType']].describe()

,channelGrouping,socialEngagementType
count,903653,903653
unique,8,1
top,Organic Search,Not Socially Engaged
freq,381561,903653


---
**Parsing timestamp and dropping date, socialEngagementType:**

In [20]:
# CLEAN STEP 7

from datetime import datetime

def parse_dates_clean(df):
    df['visitStartTime'] = df['visitStartTime'].apply(lambda time: datetime.fromtimestamp(time))
    df['year']  = df['visitStartTime'].dt.year
    df['month'] = df['visitStartTime'].dt.month
    df['day']   = df['visitStartTime'].dt.day
    df['hour']  = df['visitStartTime'].dt.hour + (df['visitStartTime'].dt.minute/60)
    df.drop(columns=['date', 'visitStartTime', 'socialEngagementType'], inplace=True)

    return df

In [21]:
df = parse_dates_clean(df)

others = [c for c in df.columns if c not in (totals_cols + geoNetwork_cols + device_cols + trafficSource_cols)]
df[others].head()

,channelGrouping,fullVisitorId,sessionId,visitId,visitNumber,year,month,day,hour
0,Organic Search,1131660440785968503,1131660440785968503_1472830385,1472830385,1,2016,9,2,11.550000
1,Organic Search,377306020877927890,377306020877927890_1472880147,1472880147,1,2016,9,3,1.366667
2,Organic Search,3895546263509774583,3895546263509774583_1472865386,1472865386,1,2016,9,2,21.266667
3,Organic Search,4763447161404445595,4763447161404445595_1472881213,1472881213,1,2016,9,3,1.666667
4,Organic Search,27294437909732085,27294437909732085_1472822600,1472822600,2,2016,9,2,9.383333


---
## Clean Pipeline

In [22]:
def clean(df):
    df = parse_json_cols(df)
    df = cast_totals(df)
    df = drop_visits(df)
    df = drop_geo_cols(df)
    df = drop_device_cols(df)
    df = drop_trafficSource_cols(df)
    df = parse_dates_clean(df)
    
    return df

## Do the same for test dataset

In [23]:
tdf = pd.read_csv('./all/test.csv', dtype={'fullVisitorId': str})
tdf = clean(tdf)
tdf.head()

,channelGrouping,fullVisitorId,sessionId,visitId,visitNumber,device/browser,device/deviceCategory,device/isMobile,device/operatingSystem,geoNetwork/continent,...,trafficSource/adwordsClickInfo/page,trafficSource/adwordsClickInfo/slot,trafficSource/isTrueDirect,trafficSource/medium,trafficSource/referralPath,trafficSource/source,year,month,day,hour
0,Organic Search,6167871330617112363,6167871330617112363_1508151024,1508151024,2,Chrome,desktop,False,Macintosh,Asia,...,NaN,NaN,True,organic,NaN,google,2017,10,16,6.833333
1,Organic Search,0643697640977915618,0643697640977915618_1508175522,1508175522,1,Chrome,desktop,False,Windows,Europe,...,NaN,NaN,NaN,organic,NaN,google,2017,10,16,13.633333
2,Organic Search,6059383810968229466,6059383810968229466_1508143220,1508143220,1,Chrome,desktop,False,Macintosh,Europe,...,NaN,NaN,NaN,organic,NaN,google,2017,10,16,4.666667
3,Organic Search,2376720078563423631,2376720078563423631_1508193530,1508193530,1,Safari,mobile,True,iOS,Americas,...,NaN,NaN,NaN,organic,NaN,google,2017,10,16,18.633333
4,Organic Search,2314544520795440038,2314544520795440038_1508217442,1508217442,1,Safari,desktop,False,Macintosh,Americas,...,NaN,NaN,NaN,organic,NaN,google,2017,10,17,1.283333


## Save both clean dataframes

In [24]:
columns = list(df.columns)
columns.remove('totals/transactionRevenue')
columns.append('totals/transactionRevenue')
df = df[columns[::-1]]
df.to_csv('./all/clean_train.csv')

df.to_csv('./all/clean_train.csv', index=False)
tdf.to_csv('./all/clean_tdf.csv', index=False)